In [20]:
import pandas as pd
from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts, load_highcharts
from pandas.compat import StringIO

from IPython.core.display import display, HTML

def show(div, js):
    display(HTML("""
    <div id="%s"</div>
    <script type="text/javascript">%s</script>
    """ % (div, js)))

load_highcharts()

In [8]:
dat = """ts;A;B;C
2015-01-01 00:00:00;27451873;29956800;113
2015-01-01 01:00:00;20259882;17906600;76
2015-01-01 02:00:00;11592256;12311600;48
2015-01-01 03:00:00;11795562;11750100;50
2015-01-01 04:00:00;9396718;10203900;43
2015-01-01 05:00:00;14902826;14341100;53"""
df = pd.read_csv(StringIO(dat), sep=';', index_col='ts', parse_dates=['ts'])

In [9]:
df

,A,B,C
ts,,,
2015-01-01 00:00:00,27451873,29956800,113
2015-01-01 01:00:00,20259882,17906600,76
2015-01-01 02:00:00,11592256,12311600,48
2015-01-01 03:00:00,11795562,11750100,50
2015-01-01 04:00:00,9396718,10203900,43
2015-01-01 05:00:00,14902826,14341100,53


In [23]:
# Basic line plot
chart = serialize(df, render_to="my-chart1", title="My Chart")
show("my-chart1", chart)

In [24]:
# Basic column plot
chart = serialize(df, render_to="my-chart2", title="Test", kind="bar")
show("my-chart2", chart)

In [25]:
# Basic column plot
chart = serialize(df, render_to="my-chart3", title="Test", kind="barh")
show("my-chart3", chart)

In [26]:
# Plot C on secondary axis
chart = serialize(df, render_to="my-chart4", title="Test", secondary_y = ["C"])
show("my-chart4", chart)

In [27]:
# Plot on a 1000x700 div
chart = serialize(df, render_to="my-chart5", title="Test", figsize = (1000, 700))
show("my-chart5", chart)

In [29]:
class Test(object):
    def __init__(self, x=None, y=None, z=24):
        print(locals())
Test(1,2)

{'x': 1, 'z': 24, 'y': 2, 'self': <__main__.Test object at 0x10c67b6d8>}


In [460]:
from traitlets import HasTraits, Unicode, Integer, Bool, List, Tuple, Float, Any, TraitType, Instance, Type, Undefined

from uuid import uuid4
from hcplot.utils.color import web2rgba, css3Colors
class HCBase(HasTraits):

    def toDict(self):
        result = {}
        for key, cls in self.class_traits().items():
            value = getattr(self, key)
            if isinstance(value, HCBase):
                result[key] = value.toDict()
            elif isinstance(value, (list,tuple)):
                tmp = [el.toDict() if isinstance(el, HCBase) else el for el in value]
                if len(tmp) > 0:
                    result[key] = tmp 
            else:
                if value is not None and value != [] and value != {}:
                    if hasattr(cls, "conv"):
                        result[key] = cls.conv(value)
                    else:
                        result[key] = value
        return result

In [509]:

class Color(TraitType):
    """A color trait."""

    default_value = "rgba(124,181,236,1.0)"
    info_text = 'an rgba color'
    
    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if isinstance(value, tuple) and (len(value) == 3 or len(value) == 4):
            return value
        else:
            try:
                rgba = web2rgba(value)
                return rgba
            except Exception:
                self.error(obj, value)

    @staticmethod
    def conv(value):
        return "rgba(%d,%d,%d,%5.3f)" % value
    

class Function(TraitType):
    """A Javascript Function trait."""

    default_value = "function() {}"
    info_text = 'string representation of a javascript function'
    
    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if isinstance(value, str) and value.startswith("function()"):
            return value
        else:
            self.error(obj, value)


class CSSObject(TraitType):
    """A CSS trait."""

    default_value = {}
    info_text = 'a CSS dict {"key": value, ...}'
    
    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if isinstance(value, dict):
            return value
        else:
            self.error(obj, value)
    
    
class Shadow(HCBase):
    color = Color(None, allow_none=True)
    offsetX = Integer(None, allow_none=True)
    offsetY = Integer(None, allow_none=True)
    opacity = Float(None, allow_none=True)
    width = Integer(None, allow_none=True)
    

class _Shadow(TraitType):
    default_value = True
    info_text = 'Boolean or an Shadow object'
    
    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if type(value) == type(True) or isinstance(value, Shadow):
            return value
        else:
            self.error(obj, value)

class Animation(HCBase):
    duration = Float(None, allow_none=True)
    easing = Unicode(None, allow_none=True)

    
class _Animation(TraitType):
    default_value = True
    info_text = 'Boolean or an Animation object'
    
    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if type(value) == type(True) or isinstance(value, Animation):
            return value
        else:
            self.error(obj, value)


class BooleanOrString(TraitType):
    default_value = True
    info_text = 'Boolean or String'
    
    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if type(value) == type(True) or isinstance(value, str):
            return value
        else:
            self.error(obj, value)


class NumberOrString(TraitType):
    default_value = True
    info_text = 'Number or String'
    
    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if type(value) in [int, float] or isinstance(value, str):
            return value
        else:
            self.error(obj, value)


class IntegerOrPercentageString(TraitType):
    default_value = "50%"
    info_text = 'Integer or PercentageString("50%")'
    
    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if type(value) == int or (type(value) == str  and value[-1] == "%"):
            return value
        else:
            self.error(obj, value)
            
            
class XYFloat(HCBase):
    x = Float(None, allow_none=True)
    y = Float(None, allow_none=True)


class XYInteger(HCBase):
    x = Integer(None, allow_none=True)
    y = Integer(None, allow_none=True)


class Position(XYFloat):
    align = Unicode(None, allow_none=True)
    verticalAlign = Unicode(None, allow_none=True)

    def __init__(self, **kwargs):
        super(__class__, self).__init__(**kwargs)    

        
class SubTitle(XYInteger):
    align = Unicode(None, allow_none=True)
    floating = Bool(None, allow_none=True)
    style = CSSObject(None, allow_none=True)
    text = Unicode(None, allow_none=True)
    useHTML = Bool(None, allow_none=True)
    verticalAlign = Unicode(None, allow_none=True)
    widthAdjust = Integer(None, allow_none=True)


class Title(SubTitle):
    def __init__(self, **kwargs):
        super(__class__, self).__init__(**kwargs)    

class AxisStop2(TraitType):
    default_value = ()
    info_text = 'Tuple (Float, Color)'
    
    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        print(value)
        if isinstance(value, (tuple, list)) and len(value) == 2 and \
           isinstance(value[0], float) and isinstance(value[1], Color):
            return value
        else:
            self.error(obj, value)    
    
class AxisStop(HCBase):
    stop = Float(allow_none=False)
    color = Color(allow_none=False)


class _AxisStop(TraitType):
    default_value = ()
    info_text = 'AxisStop object'
    
    def __init__(self, default_value=Undefined, allow_none=False, **kwargs):
        super(__class__, self).__init__(default_value=default_value,
                                        allow_none=allow_none, **kwargs)

    def validate(self, obj, value):
        if isinstance(value, AxisStop):
            return value
        else:
            self.error(obj, value)     

    @staticmethod
    def conv(value):
        return (value.stop, value.color)
        
        
class Marker(XYFloat):
    
    color = Color(None, allow_none=True)
    position = Instance(Position, allow_none=True)
    name = Unicode(None, allow_none=True)
    animation = _Animation(None, allow_none=True)
    style = CSSObject(None, allow_none=True)
    click = Function(None, allow_none=True)
    title = Instance(Title, allow_none=True)
    subtitle = Instance(SubTitle, allow_none=True)
    shadow = _Shadow(None, allow_none=True)
    test = BooleanOrString(None, allow_none=True)
    width = IntegerOrPercentageString(None, allow_none=True)
    center = List(IntegerOrPercentageString(), maxlen=2)
    positions = List(Instance(Position))
    stops = List(Tuple(Float(), Color()))
    col = List(List(NumberOrString(None, allow_none=True)))
    
    def __init__(self, **kwargs):
        super(__class__, self).__init__(**kwargs)
        
        

In [514]:
m = Marker(color="Red", 
           x=4.2, 
           position=Position(x=1, y=2), 
           animation=Animation(duration=0.5),
           subtitle=SubTitle(x=10, y=20), 
           style={"width": "10px"}, 
           click="function(){console.log(\"yep\")}",
           shadow=Shadow(color="Blue"),
           test=True,
           width=23,
           center=(20, "50%"),
           stops=[(0.1, "Red"), (0.2, "Green")],
           positions=[Position(x=0, align="top")],
           col=[
            [None, 'Apples', 'Pears', 'Oranges'],
            ['Ola', 1, 4, 3],
            ['Kari', 5, 4, 2]
            ]
          )
m.toDict()

{'animation': {'duration': 0.5},
 'center': [20, '50%'],
 'click': 'function(){console.log("yep")}',
 'col': [[None, 'Apples', 'Pears', 'Oranges'],
  ['Ola', 1, 4, 3],
  ['Kari', 5, 4, 2]],
 'color': [255, 0, 0, 1.0],
 'position': {'x': 1.0, 'y': 2.0},
 'positions': [{'align': 'top', 'x': 0.0}],
 'shadow': {'color': [0, 0, 255, 1.0]},
 'stops': [(0.1, (255, 0, 0, 1.0)), (0.2, (0, 128, 0, 1.0))],
 'style': {'width': '10px'},
 'subtitle': {'x': 10, 'y': 20},
 'test': True,
 'width': 23,
 'x': 4.2}

In [484]:
color = Color("#abc")

In [124]:
Position(x=1, y=2).toDict()

{}

TraitError: The 'x' trait of a Position instance must be an int, but a value of 1.2 <class 'float'> was specified.

In [118]:
from traitlets.config.configurable import Configurable
from traitlets import Integer, Float, Unicode, Bool

class MyClass(Configurable):
    name = Unicode(u'defaultname', help="the name of the object").tag(config=True)
    ranking = Integer(0, help="the class's ranking").tag(config=True)
    value = Float(99.0)

In [119]:
c = MyClass()

In [124]:
c.name = "23"

In [125]:
c.name

'23'

In [126]:
c2 = MyClass(name="wert", ranking=5, value=42.0)

In [127]:
c2.value

42.0

In [129]:
dir(c2)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_notifiers',
 '_config_changed',
 '_cross_validation_lock',
 '_find_my_config',
 '_load_config',
 '_notify_trait',
 '_register_validator',
 '_remove_notifiers',
 '_trait_default_generators',
 '_trait_notifiers',
 '_trait_validators',
 '_trait_values',
 'add_traits',
 'class_config_rst_doc',
 'class_config_section',
 'class_get_help',
 'class_get_trait_help',
 'class_own_trait_events',
 'class_own_traits',
 'class_print_help',
 'class_trait_names',
 'class_traits',
 'config',
 'cross_validation_lock',
 'has_trait',
 'hold_trait_notifications',
 'name',
 'notify_change',
 'observe',
 'on_trait_change',
 'pa